### 160927 출장시 내 디바이스 찾기...

Date: 2016-09-28<br>
Writer: Sundong Kim<br>

16년 09월 27일 코오롱스포츠 문정직영점 출장 때 디바이스를 4개 가지고 갔었다. (핸드폰, 아이패드1미니, 갤럭시탭 2개)
방문한 시각: 13시 13분 경 을 기준으로 하여 4개 디바이스의 trajectory를 가지는지 판단하고자 하여 이 note를 만듦.

사실 핸드폰은 계속 들고 다녔고(와이파이를 일부러 껐다 켰다 해봄), 패드들은 가방 안에 같이 보관되어 있었음.


### 결론
160928: 아직은 내 device 를찾지 못하였다...
160929: device를 찾았다. deny=True에 존재했다.


### 찾기 위한 노력
조이(ZOYI - Wi-Fi 수신기 제조업체 1644-4052)에 전화: ZOYI Wi-Fi 수신기를 이용하는 코오롱 매장 데이터를 분석하고 있는 카이스트 xxx 입니다. API를 통해 제공되는 json형식의 파일을 이해하는데 도움이 필요한데 담당자와 연결시켜 주실 수 있으신가요? 

담당자분이 직접 핸드폰으로 연락주신다고 하심 (통화: Analyst 유인제씨 - iron@zoyi.yo, 010-2726-2790와 통화)

0. 소속 말하기 - 카이스트 산업공학과 이재길교수님 연구실 박사과정 김선동입니다. 
1. 매장 내부의 움직임 분석해서 고객들의 재방문을 의도하는가 연구를 진행중입니다.
2. 현재 분석하고 있는 데이터는 코오롱 문정점인데, json형식의 데이터를 이해하는데 도움이 필요해서 전화를 드렸다.
3. 코오롱 방문해 보니, zoyi wi-fi 기계로 방문자들이 와이파이를 사용하는건 아니고 트래킹 용인거 같은데, dwell_time이 어떻게 정의되는 지 궁금: Wi-Fi AP와 수신 가능한 상태 시간 을 의미하는 건지 궁금. 특정 시점에 한 군데의 Wi-Fi 수신기와 접속중이 아니라 여러 군데에 연결 가능한 상태일 수 있다
    * 센서에 인터넷이 연결되지 않아도 수집을 수집할 수 있다. 
    * Dwell time: 센서가 맨 처음 수집한 시간과 맨 마지막 시간의 차이. 
4. device_id가 핸드폰 맥 어드레스라고 들었습니다, 핸드폰의 Wi-Fi 맥어드레스는 C4:9A:33:1A:23:FT 이런식으로 생겼는데, 이것과 '4cfef2077e81acfb7e8ed5f266408f7e' 이런 형식의 데이터가 일치하지 않아서 여기에 다른 인코딩 등이 들어가나 궁금합니다.
    * 맥 어드레스는 개인 정보이기 때문에, 수집한 후 바로 hash화 시켜서 저장함. 로직은 복호화시키는 과정이기 때문에 말할 수는 없다. 다만 하나의 맥 어드레스는 하나의 device id이다.
5. Wi-Fi 수신기가 세부 구역별로(1층에 3개, 2층에 3개, 총 6개) 설치되어 있던데, 로그에 찍히는 구역은 out, in, 1f, 2f, 3f, 1f세부 3개, 2f세부 3개  - 매장 전체를 아우르는 in, 유동인구: 센서하나에서 여러 가지 크기의 영역 반지름에 따른  원의 크기. out는 여러 대에 
    * 각 센서는 여러 거리별로(예: 1m, 2m, 3m) 디바이스와 접촉할 수 있고, 그에 따라 세부존, 중간존, 매장 출입, 유동인구에 대한 분석을 수행한다. 
    * 세부 존: 1f inner 센서가 device와 처음 접촉한 순간이 ts, (접촉이 끝난 순간 - 처음 접촉한 순간)이 dwell_time.
    * 좀더 큰 존: 1f의 경우에는 3개의 센서가 특정 거리에 대해 처음 디텍팅된 시간부터 마무리된 순간까지를 1f로 나타냄
    * in과 out: 모든 센서들을 통틀어 분석하는데, 매장 입구에 해당되는 거리 이상에 닿은 순간부터 떨어진 순간까지를 in으로 판단, out의 경우는 유동 인구 체크하려고 좀더 거리를 넓혀서 같은 분석을 수행
6. 직원 deny=True 한번 직원은 영원한 직원은 아님? 
    * deny의 경우 매일 매일 재계산을 함.
    * 로직: deny=True인 건, 2주동안 10회 이상 방문 or 한번 방문했을 때 체류시간이 1시간 이상. 그러면 deny라고 설정해준다. 주변 매장 직원의 경우에도 out 센서에 접촉 가능한 빈도가 2주동안 10회 이상이면 deny로 체크
7. 메일로 웬만한건 물어보면 될 듯.(유인재: iron@zoyi.co), 방문은 지양하나 필요하다면 스케쥴을 확인해봄 



최근 데이터 소량 크롤링(crawler.ipynb 참고하여 만듦)

In [3]:
### import libraries
import urllib.request, json, re
import pandas as pd
import datetime
import time, calendar

In [4]:
urlstrip1 = "http://walkinsights.com/api/v1/shops/786/sessions?"
urlstrip2 ="limit=1000&user_token=77eFJPB37UkgYyLjWZrx&user_email=jeeyeachung@kolon.com"
url = urlstrip1 + urlstrip2
response = urllib.request.urlopen(url)
str_response = response.read().decode('utf-8')
obj = json.loads(str_response)

In [5]:
ts = obj['since']
ts

'7fea88f6d93e4f9c:2f-inner'

In [4]:
with open('../data/786_160928_sample/786_'+ts+'.json', 'w') as f:
    json.dump(obj, f)
f.close()

In [6]:
while(True):
    urlstrip3 = "since="+ts+"&limit=1000&"
    url = urlstrip1 + urlstrip3 + urlstrip2
    response = urllib.request.urlopen(url)
    str_response = response.read().decode('utf-8')
    obj2 = json.loads(str_response)
    filename = '../data/786_160928_sample/786_'+ts+'.json'
    with open(filename, 'w') as f:
        json.dump(obj2, f)
    f.close()
    ts = ts = obj2['since']

In [2]:
import glob
import os

In [8]:
contents = []
files = glob.glob('../data/786_160928_sample/*.json')
for filename in files:
    with open(filename, 'r') as f:
        d = json.load(f)
        contents.extend(d['sessions'])

df = pd.DataFrame(contents)

In [9]:
df.to_pickle("../data/786_160928_sample/786_160928_sample.p")

Read & Analyze

In [6]:
### import libraries
import pandas as pd
import datetime
import numpy as np
import re

In [38]:
df = pd.read_pickle("../data/786_160928_sample/786_160928_sample.p")
df.head(5)   ### Show raw data

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts
0,out,NaN,b28360ee425bc932af435775834b60f7,0,786:7fea89496eff1d23:out,NaN,NaN,1474975830030
1,out,NaN,736652b63d4cff1ce9648fd19c195b1e,282,786:7fea89496fbc7c1c:out,NaN,NaN,1474975827000
2,out,NaN,b14b0a69235292dd4d09f75f19079bed,38,786:7fea894970a96da6:out,NaN,NaN,1474975823209
3,out,NaN,b59aa37cc2e30b17c1a3617a442f2f78,39,786:7fea894970bdadfa:out,NaN,NaN,1474975822885
4,out,NaN,4cfef2077e81acfb7e8ed5f266408f7e,14,786:7fea894971d33cf5:out,NaN,NaN,1474975818444


In [39]:
df['date'] = df['ts'] // 86400000
# df = df.loc[(df['deny']!= True) & (df['dwell_time'] > 0)]
df = df.loc[(df['dwell_time'] > 0)]
df['date_device_id'] = df.date.map(str) + "_" + df.device_id
df.head(5)

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id
1,out,NaN,736652b63d4cff1ce9648fd19c195b1e,282,786:7fea89496fbc7c1c:out,NaN,NaN,1474975827000,17071,17071_736652b63d4cff1ce9648fd19c195b1e
2,out,NaN,b14b0a69235292dd4d09f75f19079bed,38,786:7fea894970a96da6:out,NaN,NaN,1474975823209,17071,17071_b14b0a69235292dd4d09f75f19079bed
3,out,NaN,b59aa37cc2e30b17c1a3617a442f2f78,39,786:7fea894970bdadfa:out,NaN,NaN,1474975822885,17071,17071_b59aa37cc2e30b17c1a3617a442f2f78
4,out,NaN,4cfef2077e81acfb7e8ed5f266408f7e,14,786:7fea894971d33cf5:out,NaN,NaN,1474975818444,17071,17071_4cfef2077e81acfb7e8ed5f266408f7e
8,out,NaN,54cdaa508cfe85aaa6a9f022d612ce23,3,786:7fea894974f2ae56:out,NaN,NaN,1474975805653,17071,17071_54cdaa508cfe85aaa6a9f022d612ce23


In [40]:
# ### deny = 0은 걍 포함 (류인제씨 통화 이후)


# ### 직원 표시가 한번이라도 있는 device_id 삭제 (dwell_time > 0인 것들 중에서 직원 표시가 있는 id만 삭제. 
# ### dwell_time = 0인 로그가 왜이리 많은지.

# worker = set(df.loc[df['deny']==True]['device_id'].unique().tolist()) ### deny=True가 있는 user id의 set

# df = df[-df.device_id.isin(worker)] ### worker가 아닌 customer들만 df에서 남김.

In [41]:
df.groupby(['date_device_id'])['area']

In [42]:
traj = df.groupby(['date_device_id'])['area']
ar = ['out']
temp = traj.unique()
temp2 = temp[-temp.apply(lambda x: (x == ar).all())]

In [43]:
df_sample = df[df.date_device_id.isin(temp2.index.tolist())]

In [44]:
mplogs = df_sample.sort_index(ascending=False).groupby('date_device_id', sort=False).apply(lambda x: x.index.tolist()).sort_index()

In [45]:
mpframe = mplogs.to_frame(name='logs').reset_index()

In [46]:
def finddata(logs, df, data):   # x is a list of logs
   return df.ix[logs][data].tolist()

In [47]:
mpframe['traj'] = mpframe['logs'].apply(lambda x: finddata(x, df, 'area'))
mpframe['deny'] = mpframe['logs'].apply(lambda x: finddata(x, df, 'deny'))

In [48]:
df.shape

(24849, 10)

In [49]:
mpframe['ts_area'] = mpframe['logs'].apply(lambda x: finddata(x, df, 'ts'))
mpframe['dwelltime_area'] = mpframe['logs'].apply(lambda x: finddata(x, df, 'dwell_time'))

In [50]:
mpframe['ts_area'] = mpframe['ts_area'].apply(lambda x: [int(y/1000) for y in x])

In [51]:
mpframe.shape

(357, 6)

In [52]:
mpframe.ts_area.head(5)

0                [1474623592, 1474623776, 1474623776]
1    [1474614988, 1474615015, 1474615015, 1474615015]
2                            [1474620743, 1474621458]
3                [1474620716, 1474620926, 1474620926]
4    [1474616922, 1474616942, 1474617036, 1474617042]
Name: ts_area, dtype: object

In [53]:
mpframe['hour_start'] = mpframe['ts_area'].apply(lambda x: [int(datetime.datetime.fromtimestamp(y).strftime('%H')) for y in x])
mpframe['time_start'] = mpframe['ts_area'].apply(lambda x: [datetime.datetime.fromtimestamp(y).strftime('%H:%M:%S') for y in x])  # %Y-%m-%d 

In [54]:
mpframe.head(1)

,date_device_id,logs,traj,deny,ts_area,dwelltime_area,hour_start,time_start
0,17067_0d30577f479ccb6f0d65e3c5f71c7c1f,"[47207, 47157, 47156]","[out, in, 2f]","[nan, nan, nan]","[1474623592, 1474623776, 1474623776]","[486, 115, 91]","[18, 18, 18]","[18:39:52, 18:42:56, 18:42:56]"


In [55]:
def calend(x, y):
    return list(map(lambda i, j: i+j, x, y ))

In [56]:
mpframe['ts_area_end'] = mpframe[['ts_area', 'dwelltime_area']].apply(lambda x: calend(*x), axis=1)

In [57]:
mpframe['hour_end'] = mpframe['ts_area_end'].apply(lambda x: [int(datetime.datetime.fromtimestamp(y).strftime('%H')) for y in x])
mpframe['time_end'] = mpframe['ts_area_end'].apply(lambda x: [datetime.datetime.fromtimestamp(y).strftime('%H:%M:%S') for y in x])

In [58]:
def printall(mp, i, columnss):
    for column in columnss:
        print(mp[column].ix[i])

In [59]:
printall(mpframe, 2, ['traj', 'ts_area','ts_area_end'])

['out', 'in']
[1474620743, 1474621458]
[1474621565, 1474621490]


In [60]:
mpframe.to_pickle("../data/786_160928_sample/786_160928_sample_mpframe_160923.p")

In [156]:
mpframe = pd.read_pickle("../data/786_160928_sample/786_160928_sample_mpframe_160923.p")
mpframe.head(5)   ### Show raw data

,date_device_id,logs,traj,deny,ts_area,dwelltime_area,hour_start,time_start,ts_area_end,hour_end,time_end
0,17067_0d30577f479ccb6f0d65e3c5f71c7c1f,"[47207, 47157, 47156]","[out, in, 2f]","[nan, nan, nan]","[1474623592, 1474623776, 1474623776]","[486, 115, 91]","[18, 18, 18]","[18:39:52, 18:42:56, 18:42:56]","[1474624078, 1474623891, 1474623867]","[18, 18, 18]","[18:47:58, 18:44:51, 18:44:27]"
1,17067_105a0589135736b802c07127537e9756,"[49748, 49739, 49738, 49737]","[out, in, 1f, 1f-right]","[nan, nan, nan, nan]","[1474614988, 1474615015, 1474615015, 1474615015]","[74, 16, 6, 6]","[16, 16, 16, 16]","[16:16:28, 16:16:55, 16:16:55, 16:16:55]","[1474615062, 1474615031, 1474615021, 1474615021]","[16, 16, 16, 16]","[16:17:42, 16:17:11, 16:17:01, 16:17:01]"
2,17067_156d7fa0cc69fdb76d1713142b9d290a,"[48044, 47846]","[out, in]","[nan, nan]","[1474620743, 1474621458]","[822, 32]","[17, 18]","[17:52:23, 18:04:18]","[1474621565, 1474621490]","[18, 18]","[18:06:05, 18:04:50]"
3,17067_1b1ccdcd3a14080ab4b14e270c37d73f,"[48048, 47993, 47992]","[out, in, 1f]","[nan, nan, nan]","[1474620716, 1474620926, 1474620926]","[2064, 467, 91]","[17, 17, 17]","[17:51:56, 17:55:26, 17:55:26]","[1474622780, 1474621393, 1474621017]","[18, 18, 17]","[18:26:20, 18:03:13, 17:56:57]"
4,17067_24c5ba4cceec281fef51da37e141d815,"[49126, 49118, 49095, 49094]","[out, in, 1f, 1f-right]","[nan, nan, nan, nan]","[1474616922, 1474616942, 1474617036, 1474617042]","[488, 345, 251, 245]","[16, 16, 16, 16]","[16:48:42, 16:49:02, 16:50:36, 16:50:42]","[1474617410, 1474617287, 1474617287, 1474617287]","[16, 16, 16, 16]","[16:56:50, 16:54:47, 16:54:47, 16:54:47]"


In [157]:
mpframe

,date_device_id,logs,traj,deny,ts_area,dwelltime_area,hour_start,time_start,ts_area_end,hour_end,time_end
0,17067_0d30577f479ccb6f0d65e3c5f71c7c1f,"[47207, 47157, 47156]","[out, in, 2f]","[nan, nan, nan]","[1474623592, 1474623776, 1474623776]","[486, 115, 91]","[18, 18, 18]","[18:39:52, 18:42:56, 18:42:56]","[1474624078, 1474623891, 1474623867]","[18, 18, 18]","[18:47:58, 18:44:51, 18:44:27]"
1,17067_105a0589135736b802c07127537e9756,"[49748, 49739, 49738, 49737]","[out, in, 1f, 1f-right]","[nan, nan, nan, nan]","[1474614988, 1474615015, 1474615015, 1474615015]","[74, 16, 6, 6]","[16, 16, 16, 16]","[16:16:28, 16:16:55, 16:16:55, 16:16:55]","[1474615062, 1474615031, 1474615021, 1474615021]","[16, 16, 16, 16]","[16:17:42, 16:17:11, 16:17:01, 16:17:01]"
2,17067_156d7fa0cc69fdb76d1713142b9d290a,"[48044, 47846]","[out, in]","[nan, nan]","[1474620743, 1474621458]","[822, 32]","[17, 18]","[17:52:23, 18:04:18]","[1474621565, 1474621490]","[18, 18]","[18:06:05, 18:04:50]"
3,17067_1b1ccdcd3a14080ab4b14e270c37d73f,"[48048, 47993, 47992]","[out, in, 1f]","[nan, nan, nan]","[1474620716, 1474620926, 1474620926]","[2064, 467, 91]","[17, 17, 17]","[17:51:56, 17:55:26, 17:55:26]","[1474622780, 1474621393, 1474621017]","[18, 18, 17]","[18:26:20, 18:03:13, 17:56:57]"
4,17067_24c5ba4cceec281fef51da37e141d815,"[49126, 49118, 49095, 49094]","[out, in, 1f, 1f-right]","[nan, nan, nan, nan]","[1474616922, 1474616942, 1474617036, 1474617042]","[488, 345, 251, 245]","[16, 16, 16, 16]","[16:48:42, 16:49:02, 16:50:36, 16:50:42]","[1474617410, 1474617287, 1474617287, 1474617287]","[16, 16, 16, 16]","[16:56:50, 16:54:47, 16:54:47, 16:54:47]"
5,17067_34f95176a5712ab935f65c12bdb0ea52,"[49995, 49994, 49993, 49992, 48956]","[in, 1f, 1f-inner, 1f-left, out]","[nan, nan, nan, nan, nan]","[1474613927, 1474613927, 1474613928, 147461392...","[92, 92, 92, 45, 49]","[15, 15, 15, 15, 16]","[15:58:47, 15:58:47, 15:58:48, 15:58:48, 16:59...","[1474614019, 1474614019, 1474614020, 147461397...","[16, 16, 16, 15, 17]","[16:00:19, 16:00:19, 16:00:20, 15:59:33, 17:00..."
6,17067_3c3ea325d87f622730b1628e60cf6db6,"[49176, 49105]","[out, in]","[nan, nan]","[1474616746, 1474616979]","[282, 49]","[16, 16]","[16:45:46, 16:49:39]","[1474617028, 1474617028]","[16, 16]","[16:50:28, 16:50:28]"
7,17067_3cb3c5d41981dcf7f75ffc1db3378c6e,"[49517, 48934, 48589, 47539, 46971, 46624, 461...","[1f-right, 1f-right, 1f-right, 1f-right, 1f-in...","[True, True, True, True, True, True, True, Tru...","[1474615808, 1474617630, 1474618845, 147462248...","[905, 300, 1509, 905, 300, 612, 5176, 313, 861...","[16, 17, 17, 18, 18, 19, 19, 20, 0, 5, 5, 5, 5...","[16:30:08, 17:00:30, 17:20:45, 18:21:28, 18:52...","[1474616713, 1474617930, 1474620354, 147462339...","[16, 17, 17, 18, 18, 19, 21, 21, 23, 7, 5, 7, ...","[16:45:13, 17:05:30, 17:45:54, 18:36:33, 18:57..."
8,17067_405fcf5e6fdc792d1d216cf4a026a810,"[49768, 49700, 48698, 48629, 48499, 46454, 463...","[2f-inner, 2f-left, 2f, 2f-left, 2f-inner, 2f,...","[True, True, True, True, True, True, True, Tru...","[1474614893, 1474615166, 1474618469, 147461873...","[1946, 825, 6614, 1222, 751, 5693, 523, 688, 4...","[16, 16, 17, 17, 17, 19, 19, 19, 8, 8, 8]","[16:14:53, 16:19:26, 17:14:29, 17:18:58, 17:26...","[1474616839, 1474615991, 1474625083, 147461996...","[16, 16, 19, 17, 17, 20, 19, 19, 21, 10, 10]","[16:47:19, 16:33:11, 19:04:43, 17:39:20, 17:39..."
9,17067_47254b16635711f6b0efb88a82290e37,"[49535, 49533, 49193, 49168, 49167, 47813, 478...","[in, 1f, out, in, 1f, in, 1f, in, 1f, in, 1f, ...","[True, True, True, True, True, True, True, Tru...","[1474615752, 1474615752, 1474616713, 147461677...","[11, 11, 26051, 6, 6, 176, 176, 381, 381, 1776...","[16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 1...","[16:29:12, 16:29:12, 16:45:13, 16:46:10, 16:46...","[1474615763, 1474615763, 1474642764, 147461677...","[16, 16, 23, 16, 16, 18, 18, 18, 18, 19, 19, 2...","[16:29:23, 16:29:23, 23:59:24, 16:46:16, 16:46..."


In [158]:
mpframe.traj

0                                          [out, in, 2f]
1                                [out, in, 1f, 1f-right]
2                                              [out, in]
3                                          [out, in, 1f]
4                                [out, in, 1f, 1f-right]
5                       [in, 1f, 1f-inner, 1f-left, out]
6                                              [out, in]
7      [1f-right, 1f-right, 1f-right, 1f-right, 1f-in...
8      [2f-inner, 2f-left, 2f, 2f-left, 2f-inner, 2f,...
9      [in, 1f, out, in, 1f, in, 1f, in, 1f, in, 1f, ...
10     [in, 2f-inner, 2f, in, 2f-inner, 2f, out, out,...
11                               [out, in, 1f, 1f-right]
12                                     [out, in, 2f, 1f]
13                                        [out, in, out]
14                                             [out, in]
15        [out, in, 2f, 2f-right, 2f-left, 1f, 1f-right]
16                                             [out, in]
17     [3f, in, 3f, in, 3f, 3f,

In [179]:
mpframe[mpframe.date_device_id.str.startswith('17071')].date_device_id  ## 9월 27일 데이터

303    17071_07efe7f0724f4ade616c11768258cb1c
304    17071_0b435eaa07b9373f05637cf8b03a8da2
305    17071_0e285cca09c99b114c1adf1a8fc2fac2
306    17071_123fd393d9c0f7b2ed285a64c509f6b0
307    17071_158c04751bcda6dd0326fdb5bb210634
308    17071_1c987aac0e0a0ad4e6c4fdcbf76308de
309    17071_24cf9482edc593eb79ec7ab81e50b8a5
310    17071_268eac5b638caea26f795a8fbd5da92b
311    17071_278cd36886e28d27fa03b4e7aef21687
312    17071_2f10b26efcf4f3806ee41b3337158398
313    17071_3152692afe8ff0e5715aacb89ab92ef4
314    17071_3cb3c5d41981dcf7f75ffc1db3378c6e
315    17071_3e6d2d72e4adda00d4fa3349be12c0b0
316    17071_405fcf5e6fdc792d1d216cf4a026a810
317    17071_445f102a759a807b60803c95b19a6132
318    17071_47254b16635711f6b0efb88a82290e37
319    17071_47cbc1c26d0aa075de241967ac014e51
320    17071_4c676782d37f8f3512ca0e563abc6bd6
321    17071_5550d077aac5ff2fdab5300565143d8c
322    17071_55d34cde525efb711d98c87f172dee60
323    17071_5b5232993bcc525e43d7f2b29eb23ece
324    17071_5c26eaad7fa5a2f07a75c

In [160]:
def ganttprint(mfs, i):
    
    qa = mfs['traj'].ix[i] + [1000*y for y in mfs['ts_area'].ix[i]] + [1000*z for z in mfs['ts_area_end'].ix[i]]
    l = int(len(qa)/3)
    qb = np.asarray(qa)
    shape = ( 3,l )
    qc = qb.reshape( shape ).T
    qd = pd.DataFrame(qc, columns=['Task', 'Start', 'Finish'])
    qd['Start'] = qd['Start'].astype('int') 
    qd['Finish'] = qd['Finish'].astype('int')
    qd['Complete'] = np.arange(40, 60, (60-40)/l)
    qd['t_start'] = qd['Start'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%H:%M:%S'))
    qd['t_end'] = qd['Finish'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%H:%M:%S'))
    # .as_matrix()
    return qd

In [161]:
lenlogs = np.asarray([len(x) for x in mpframe.logs])
indxx = np.argsort(lenlogs, )
largg = indxx[::-1]

In [162]:
ganttprint(mpframe, largg[13])

,Task,Start,Finish,Complete,t_start,t_end
0,in,1474762497000,1474762648000,40.000000,09:14:57,09:17:28
1,1f-right,1474762497000,1474762648000,40.909091,09:14:57,09:17:28
2,1f,1474762497000,1474762648000,41.818182,09:14:57,09:17:28
3,in,1474763690000,1474781880000,42.727273,09:34:50,14:38:00
4,1f,1474763690000,1474781880000,43.636364,09:34:50,14:38:00
5,1f-right,1474763816000,1474765457000,44.545455,09:36:56,10:04:17
6,1f-inner,1474765589000,1474766265000,45.454545,10:06:29,10:17:45
7,1f-right,1474766373000,1474766716000,46.363636,10:19:33,10:25:16
8,1f-right,1474767807000,1474768860000,47.272727,10:43:27,11:01:00
9,1f-right,1474769580000,1474770898000,48.181818,11:13:00,11:34:58


In [163]:
didcands = ['17071_5550d077aac5ff2fdab5300565143d8c', '17071_5dd63ba8a12d13f0a397a548062be9c4', '17071_8c854c38fdb2f233eb6bc6c80df6c11e', '17071_c3035b21511ce11866627fcbf181644e','17071_de12dccd21897a9d433ecfef28afdf63']

In [164]:
mpframe_cand = mpframe[mpframe.date_device_id.isin(didcands)].reset_index()  ## 9월 27일 데이터

In [165]:
mpframe_cand

,index,date_device_id,logs,traj,deny,ts_area,dwelltime_area,hour_start,time_start,ts_area_end,hour_end,time_end
0,321,17071_5550d077aac5ff2fdab5300565143d8c,"[5667, 5652, 5651, 5355, 5319, 5318, 5040, 503...","[out, in, 2f, 2f, 1f-right, 1f, in, 2f, in]","[True, True, True, True, True, True, True, Tru...","[1474949612, 1474949671, 1474949671, 147495093...","[8358, 1878, 45, 614, 210, 210, 1725, 569, 48]","[13, 13, 13, 13, 13, 13, 14, 14, 15]","[13:13:32, 13:14:31, 13:14:31, 13:35:35, 13:37...","[1474957970, 1474951549, 1474949716, 147495154...","[15, 13, 13, 13, 13, 13, 14, 14, 15]","[15:32:50, 13:45:49, 13:15:16, 13:45:49, 13:41..."
1,325,17071_5dd63ba8a12d13f0a397a548062be9c4,"[5611, 5603, 5602, 5546, 5534, 5330, 5299, 529...","[out, in, 1f, 2f, 2f-inner, 1f, 1f-right, 1f-l...","[True, True, True, True, True, True, True, Tru...","[1474949861, 1474949881, 1474949881, 147495014...","[8116, 7679, 3, 7208, 1949, 285, 158, 157, 124...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 1...","[13:17:41, 13:18:01, 13:18:01, 13:22:26, 13:23...","[1474957977, 1474957560, 1474949884, 147495735...","[15, 15, 13, 15, 13, 13, 13, 13, 14, 14, 14, 1...","[15:32:57, 15:26:00, 13:18:04, 15:22:34, 13:55..."
2,338,17071_8c854c38fdb2f233eb6bc6c80df6c11e,"[5658, 5657, 5656, 5655, 5514, 5309, 4799, 479...","[out, in, 1f, 2f, 2f-left, 1f, 2f-inner, 2f-le...","[True, True, True, True, True, True, True, Tru...","[1474949646, 1474949646, 1474949646, 147494964...","[8435, 8434, 317, 7207, 494, 951, 634, 1268, 313]","[13, 13, 13, 13, 13, 13, 14, 14, 15]","[13:14:06, 13:14:06, 13:14:06, 13:14:07, 13:24...","[1474958081, 1474958080, 1474949963, 147495685...","[15, 15, 13, 15, 13, 13, 14, 14, 15]","[15:34:41, 15:34:40, 13:19:23, 15:14:14, 13:32..."
3,349,17071_c3035b21511ce11866627fcbf181644e,"[5653, 5650, 5649, 5648, 5646, 5639, 5620, 561...","[out, in, 2f-right, 2f, 2f-left, 2f-inner, 1f,...","[True, True, True, True, True, True, True, Tru...","[1474949670, 1474949675, 1474949693, 147494969...","[8459, 8411, 75, 2322, 546, 1766, 1237, 86, 81...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[13:14:30, 13:14:35, 13:14:53, 13:14:53, 13:15...","[1474958129, 1474958086, 1474949768, 147495201...","[15, 15, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[15:35:29, 15:34:46, 13:16:08, 13:53:35, 13:24..."
4,352,17071_de12dccd21897a9d433ecfef28afdf63,"[5637, 5636, 5635, 5634, 5283, 5217, 5216]","[out, in, 1f, 1f-right, in, 1f, 1f-right]","[nan, nan, nan, nan, nan, nan, nan]","[1474949734, 1474949734, 1474949734, 147494973...","[2403, 784, 784, 764, 895, 574, 455]","[13, 13, 13, 13, 13, 13, 13]","[13:15:34, 13:15:34, 13:15:34, 13:15:35, 13:40...","[1474952137, 1474950518, 1474950518, 147495049...","[13, 13, 13, 13, 13, 13, 13]","[13:55:37, 13:28:38, 13:28:38, 13:28:19, 13:55..."


In [166]:
lenlogs = np.asarray([len(x) for x in mpframe_cand.logs])
indxx = np.argsort(lenlogs, )
largg = indxx[::-1]

In [167]:
def printchartlah(f, i):
    lenlogs = np.asarray([len(x) for x in f.logs])
    indxx = np.argsort(lenlogs, )
    largg = indxx[::-1]
    print("---------------", i,'th longest moving pattern: Sequence #', largg[i])
    print("---------------", "Date_Device_id: ", f[largg[i]:largg[i]+1].date_device_id.values[0])
    qd = ganttprint(f, largg[i])  ## .sort_index(ascending=False)
    print(qd[['Task','t_start', 't_end']])
    return qd

In [168]:
qd0 = printchartlah(mpframe_cand, 0)

--------------- 0 th longest moving pattern: Sequence # 3
--------------- Date_Device_id:  17071_c3035b21511ce11866627fcbf181644e
        Task   t_start     t_end
0        out  13:14:30  15:35:29
1         in  13:14:35  15:34:46
2   2f-right  13:14:53  13:16:08
3         2f  13:14:53  13:53:35
4    2f-left  13:15:03  13:24:09
5   2f-inner  13:15:27  13:44:53
6         1f  13:17:04  13:37:41
7   1f-right  13:17:10  13:18:36
8    1f-left  13:17:31  13:18:52
9   2f-right  13:29:02  13:36:07
10  1f-right  13:31:23  13:33:34
11  1f-inner  13:31:29  13:33:04
12   1f-left  13:31:37  13:32:22
13   2f-left  13:47:38  13:53:15
14        2f  14:24:20  15:11:03
15  2f-right  14:24:45  14:43:42
16        1f  14:30:16  14:36:30
17  2f-inner  14:35:12  14:36:30
18   2f-left  14:43:21  15:05:40
19        1f  15:00:34  15:02:44
20  1f-inner  15:00:52  15:02:44
21  2f-right  15:02:59  15:03:04
22        2f  15:24:30  15:31:15
23   2f-left  15:24:43  15:24:52
24  2f-inner  15:25:29  15:31:14
25        1f

In [169]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import datetime

In [170]:
fig0 = FF.create_gantt(qd0[['Task','Start', 'Finish', 'Complete']], colors='Viridis', index_col='Complete', show_colorbar=True)
p0 = py.iplot(fig0, filename='Numeric Variable', world_readable=True)
p0

In [171]:
qd1 = printchartlah(mpframe_cand, 1)

--------------- 1 th longest moving pattern: Sequence # 1
--------------- Date_Device_id:  17071_5dd63ba8a12d13f0a397a548062be9c4
        Task   t_start     t_end
0        out  13:17:41  15:32:57
1         in  13:18:01  15:26:00
2         1f  13:18:01  13:18:04
3         2f  13:22:26  15:22:34
4   2f-inner  13:23:13  13:55:42
5         1f  13:36:42  13:41:27
6   1f-right  13:38:49  13:41:27
7    1f-left  13:38:49  13:41:26
8    2f-left  13:49:13  14:09:53
9         1f  14:14:42  14:30:41
10   2f-left  14:35:50  14:51:30
11        1f  15:06:58  15:22:34
12   2f-left  15:11:45  15:22:34
13  2f-right  15:17:19  15:22:30


In [172]:
fig1 = FF.create_gantt(qd1[['Task','Start', 'Finish', 'Complete']], colors='Viridis', index_col='Complete', show_colorbar=True)
p1 = py.iplot(fig1, filename='Numeric Variable', world_readable=True)
p1

In [173]:
qd2 = printchartlah(mpframe_cand, 2)

--------------- 2 th longest moving pattern: Sequence # 2
--------------- Date_Device_id:  17071_8c854c38fdb2f233eb6bc6c80df6c11e
       Task   t_start     t_end
0       out  13:14:06  15:34:41
1        in  13:14:06  15:34:40
2        1f  13:14:06  13:19:23
3        2f  13:14:07  15:14:14
4   2f-left  13:24:41  13:32:55
5        1f  13:38:13  13:54:04
6  2f-inner  14:17:24  14:27:58
7   2f-left  14:17:25  14:38:33
8   2f-left  15:03:28  15:08:41


In [174]:
fig2 = FF.create_gantt(qd2[['Task','Start', 'Finish', 'Complete']], colors='Viridis', index_col='Complete', show_colorbar=True)
p2 = py.iplot(fig2, filename='Numeric Variable', world_readable=True)
p2

In [175]:
qd3 = printchartlah(mpframe_cand, 3)

--------------- 3 th longest moving pattern: Sequence # 0
--------------- Date_Device_id:  17071_5550d077aac5ff2fdab5300565143d8c
       Task   t_start     t_end
0       out  13:13:32  15:32:50
1        in  13:14:31  13:45:49
2        2f  13:14:31  13:15:16
3        2f  13:35:35  13:45:49
4  1f-right  13:37:50  13:41:20
5        1f  13:37:50  13:41:20
6        in  14:01:20  14:30:05
7        2f  14:01:20  14:10:49
8        in  15:23:46  15:24:34


In [176]:
fig3 = FF.create_gantt(qd3[['Task','Start', 'Finish', 'Complete']], colors='Viridis', index_col='Complete', show_colorbar=True)
p3 = py.iplot(fig3, filename='Numeric Variable', world_readable=True)
p3

In [154]:
qd4 = printchartlah(mpframe_cand, 4)

--------------- 4 th longest moving pattern: Sequence # 4
--------------- Date_Device_id:  17071_de12dccd21897a9d433ecfef28afdf63
       Task   t_start     t_end
0       out  13:15:34  13:55:37
1        in  13:15:34  13:28:38
2        1f  13:15:34  13:28:38
3  1f-right  13:15:35  13:28:19
4        in  13:40:43  13:55:38
5        1f  13:46:03  13:55:37
6  1f-right  13:46:03  13:53:38


In [155]:
fig4 = FF.create_gantt(qd4[['Task','Start', 'Finish', 'Complete']], colors='Viridis', index_col='Complete', show_colorbar=True)
p4 = py.iplot(fig4, filename='Numeric Variable', world_readable=True)
p4

In [47]:
df['ts_start'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df2 = df.loc[df.area != 'out']   

In [48]:
df

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,ts_start
1,out,NaN,736652b63d4cff1ce9648fd19c195b1e,282,786:7fea89496fbc7c1c:out,NaN,NaN,1474975827000,17071,17071_736652b63d4cff1ce9648fd19c195b1e,2016-09-27 20:30:27
2,out,NaN,b14b0a69235292dd4d09f75f19079bed,38,786:7fea894970a96da6:out,NaN,NaN,1474975823209,17071,17071_b14b0a69235292dd4d09f75f19079bed,2016-09-27 20:30:23
3,out,NaN,b59aa37cc2e30b17c1a3617a442f2f78,39,786:7fea894970bdadfa:out,NaN,NaN,1474975822885,17071,17071_b59aa37cc2e30b17c1a3617a442f2f78,2016-09-27 20:30:22
4,out,NaN,4cfef2077e81acfb7e8ed5f266408f7e,14,786:7fea894971d33cf5:out,NaN,NaN,1474975818444,17071,17071_4cfef2077e81acfb7e8ed5f266408f7e,2016-09-27 20:30:18
8,out,NaN,54cdaa508cfe85aaa6a9f022d612ce23,3,786:7fea894974f2ae56:out,NaN,NaN,1474975805653,17071,17071_54cdaa508cfe85aaa6a9f022d612ce23,2016-09-27 20:30:05
12,out,NaN,075643e2e352d326d07dc655d7065600,4,786:7fea8949759fbf22:out,NaN,NaN,1474975802884,17071,17071_075643e2e352d326d07dc655d7065600,2016-09-27 20:30:02
14,out,NaN,37e306833014d73ba32dec95cfe4c95a,2,786:7fea894978509ec5:out,NaN,NaN,1474975791862,17071,17071_37e306833014d73ba32dec95cfe4c95a,2016-09-27 20:29:51
16,out,NaN,ad911d62a54e9b2ffc4727f0641a48ec,2,786:7fea894978680fa5:out,NaN,NaN,1474975791487,17071,17071_ad911d62a54e9b2ffc4727f0641a48ec,2016-09-27 20:29:51
18,out,NaN,e02bbec13ec5e0fde594790c95172cb5,3,786:7fea894979905e01:out,NaN,NaN,1474975786746,17071,17071_e02bbec13ec5e0fde594790c95172cb5,2016-09-27 20:29:46
24,out,NaN,080653b7f19a3c4cbfa49946d728c846,682,786:7fea894986bdfdc0:out,NaN,NaN,1474975732768,17071,17071_080653b7f19a3c4cbfa49946d728c846,2016-09-27 20:28:52


In [74]:
i = 40
df2[i:i+50][['device_id', 'area', 'dwell_time', 'ts_start']]

,device_id,area,dwell_time,ts_start
2895,8b7c08b5c9795400d7ae2bbe8b1be410,2f-right,10,2016-09-27 16:50:21
2896,8b7c08b5c9795400d7ae2bbe8b1be410,in,10,2016-09-27 16:50:21
2970,07efe7f0724f4ade616c11768258cb1c,1f,115,2016-09-27 16:45:22
2971,07efe7f0724f4ade616c11768258cb1c,1f-right,2,2016-09-27 16:45:22
2972,07efe7f0724f4ade616c11768258cb1c,in,115,2016-09-27 16:45:22
3445,3152692afe8ff0e5715aacb89ab92ef4,1f,5,2016-09-27 16:08:30
3462,3152692afe8ff0e5715aacb89ab92ef4,in,81,2016-09-27 16:07:14
3831,4c676782d37f8f3512ca0e563abc6bd6,1f,7,2016-09-27 15:31:15
3832,4c676782d37f8f3512ca0e563abc6bd6,in,7,2016-09-27 15:31:15
4036,158c04751bcda6dd0326fdb5bb210634,1f,2,2016-09-27 15:18:33


In [ ]:
uids = df2[i:i+50][['device_id', 'area', 'dwell_time', 'ts_start']].device_id.unique()

In [73]:
df.loc[df.device_id==uids[4]]

,area,deny,device_id,dwell_time,key,revisit_count,revisit_period,ts,date,date_device_id,ts_start
4036,1f,NaN,158c04751bcda6dd0326fdb5bb210634,2,786:7fea895b4888cdc9:1f,NaN,NaN,1474957113203,17071,17071_158c04751bcda6dd0326fdb5bb210634,2016-09-27 15:18:33
4037,in,NaN,158c04751bcda6dd0326fdb5bb210634,2,786:7fea895b4888cdc9:in,1.0,22.0,1474957113203,17071,17071_158c04751bcda6dd0326fdb5bb210634,2016-09-27 15:18:33
4038,out,NaN,158c04751bcda6dd0326fdb5bb210634,60,786:7fea895b4e59ddb7:out,1.0,22.0,1474957089378,17071,17071_158c04751bcda6dd0326fdb5bb210634,2016-09-27 15:18:09
37805,out,NaN,158c04751bcda6dd0326fdb5bb210634,656,786:7fea8a51a2be9e12:out,1.0,19.0,1474698794006,17068,17068_158c04751bcda6dd0326fdb5bb210634,2016-09-24 15:33:14
38089,out,NaN,158c04751bcda6dd0326fdb5bb210634,35,786:7fea8a529c438d4e:out,1.0,19.0,1474697771975,17068,17068_158c04751bcda6dd0326fdb5bb210634,2016-09-24 15:16:11
